<h1>Oracle执行计划<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#定义" data-toc-modified-id="定义-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>定义</a></span></li><li><span><a href="#查看执行计划" data-toc-modified-id="查看执行计划-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>查看执行计划</a></span><ul class="toc-item"><li><span><a href="#Autotrace" data-toc-modified-id="Autotrace-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Autotrace</a></span></li><li><span><a href="#DBMS_XPLAN包" data-toc-modified-id="DBMS_XPLAN包-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>DBMS_XPLAN包</a></span><ul class="toc-item"><li><span><a href="#dbms_xplan.display" data-toc-modified-id="dbms_xplan.display-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>dbms_xplan.display</a></span></li><li><span><a href="#dbms_xplan.display_cursor" data-toc-modified-id="dbms_xplan.display_cursor-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>dbms_xplan.display_cursor</a></span></li><li><span><a href="#dbms_xplan.display_awr" data-toc-modified-id="dbms_xplan.display_awr-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>dbms_xplan.display_awr</a></span></li></ul></li><li><span><a href="#10046事件和10053事件" data-toc-modified-id="10046事件和10053事件-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>10046事件和10053事件</a></span><ul class="toc-item"><li><span><a href="#10046事件" data-toc-modified-id="10046事件-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>10046事件</a></span></li><li><span><a href="#10053事件" data-toc-modified-id="10053事件-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>10053事件</a></span></li></ul></li><li><span><a href="#AWR报告Statspack报告" data-toc-modified-id="AWR报告Statspack报告-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>AWR报告Statspack报告</a></span></li><li><span><a href="#一些现成的脚本" data-toc-modified-id="一些现成的脚本-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>一些现成的脚本</a></span></li></ul></li></ul></div>

## 定义

&emsp;&emsp;执行计划是一条查询语句在Oracle中的执行过程或访问路径的描述。

![执行计划示例](https://raw.githubusercontent.com/Henry-chr/Uplearning/main/Read/Oracle/knowledge/image/%E6%89%A7%E8%A1%8C%E8%AE%A1%E5%88%92%E7%A4%BA%E4%BE%8B.png)

执行计划的三个部分：
1. 目标SQL的正文、sql_id和执行计划对应的plan_hash_value
![SQL的正文](https://gitee.com/henry-chr/Uplearning/raw/main/Read/Oracle/knowledge/image/%E7%9B%AE%E6%A0%87SQL%E6%AD%A3%E6%96%87.png)

2. 执行计划主体，主要有内部执行步骤、执行顺序、谓词信息、列信息、Cardinality、Cost等
![执行计划主体](https://gitee.com/henry-chr/Uplearning/raw/main/Read/Oracle/knowledge/image/%E6%89%A7%E8%A1%8C%E8%AE%A1%E5%88%92%E4%B8%BB%E4%BD%93.png)

3. 执行计划的额外补充信息，是否动态采用（dynamic sampling）、是否Cardinality Feedback、是否SQL Profile
![额外补充信息](https://gitee.com/henry-chr/Uplearning/raw/main/Read/Oracle/knowledge/image/%E6%89%A7%E8%A1%8C%E8%AE%A1%E5%88%92%E8%A1%A5%E5%85%85%E4%BF%A1%E6%81%AF.png)

## 查看执行计划

### Autotrace

命令|输出内容|简写
:--:|:--:|:--:
SET AUTOTRACE OFF | 缺省模式，只显示SQL执行结果  |  SET AUTOT OFF
SET AUTOTRACE ON | 显示SQL结果、执行路径和资源消耗  |  SET AUTOT ON
SET AUTOTRACE TRACEONLY | 显示SQL结果数量、执行路径和资源消耗  |  SET AUTOT TRACE
SET AUTOTRACE TRACEONLY EXPLAN | 只显示SQL执行路径  |  SET AUTOT TRACE EXP
SET AUTOTRACE TRACEONLY STATISTICS | 只显示资源消耗  |  SET AUTOT TRACE STAT

### DBMS_XPLAN包

#### dbms_xplan.display

**dbms_xplan.display命令通常和explain plan命令结合使用**
```
explain plan for select * from emp;
select * from table(dbms_xplan.display());
```
PL/SQL Developer里F5快捷键的原理就是调用explain plan命令

    explain plan 命令的原理是Oracle将目标SQL所产生的执行计划写入PLAN_TABLE$
（ON COMMIT PRESERVE ROWS 的 GLOBAL TEMPORARY TABLE 即会话级临时表仅对当前会话有效）然后再通过dbms_xplan.display()命令将PLAN_TABLE$中的具体执行步骤格式化显示出来

#### dbms_xplan.display_cursor

**前提条件：SQL执行计划仍在Shared Pool中**
```
DBMS_XPLAN.DISPLAY_CURSOR(                       
 sql_id        IN  VARCHAR2  DEFAULT  NULL,      
 child_number  IN  NUMBER    DEFAULT  NULL,      
 format        IN  VARCHAR2  DEFAULT  'TYPICAL');
```
- sql_id

        指定位于库缓存执行计划中SQL语句的父游标。默认值为null。当使用默认值时当前会话的最后一条SQL语句的执行计划将被返回
        可以通过查询V$SQL 或 V$SQLAREA的SQL_ID列来获得SQL语句的SQL_ID。
    
- child_number
    
        指定父游标下子游标的序号。即指定被返回执行计划的SQL语句的子游标。默认值为0。如果为null，则sql_id所指父游标下所有子游标
        的执行计划都将被返回。
    
- format

        控制SQL语句执行计划的输出部分。
    
**拓展**

    dbms_xplan.display_cursor(null,null,'advanced') 仅用于SQLplus中查看刚执行过的SQL执行计划，在PL/SQL Developer中无法使用，因为工具在执行完SQL后还会执行其他的后台语句。*

#### dbms_xplan.display_awr

**目标SQL的执行计划被Oracle采集到AWR Reponsitory中**

<code>该方法不能显示谓词信息，是因为从V$SQL_PLAN导入AWR基表WRH$_SQL_PLAN时未将谓词字段access_predicates和filter_predicates导入</code>

### 10046事件和10053事件

#### 10046事件

&emsp;&emsp;10046事件可以得到Oracle内部执行系统解析、调用、等待、绑定变量等详细的信息，可以帮助我们解析一条/多条SQL、PL/SQL语句的运行状态，这些状态包括：Parse/Fetch/Execute三个阶段中遇到的等待事件、消耗的物理和逻辑读、CPU时间、执行计划等，还能帮我们分析一些DDL维护命令的内部工作原理，RMAN、Data Pump Expdp/impdp等工具缓慢问题。

* 执行步骤
    1. 激活10046事件

    方法一
```
alter session set events '10046 trace name context forever,level 12';
```
    方法二
```
oradebug setmypid
oradebug event 10046 trace name context forever,level 12;
```
    2. 执行目标SQL
```
select * from emp;
```
    3. 查看生成的trace文件

    方法一
```
select distinct(m.sid),p.pid,p.tracefile from v$mystat m,v$session s,v$process p where m.sid=s.sid and s.paddr=p.addr;
```
    方法二
```
oradebug tracefile_name
```
    4. 关闭10046事件

    方法一
```
alter session set events '10046 trace name context off';
```
    方法二
```
oradebug event 10046 trace name context forever,level 12;
```

    5. tkprof命令规范文件格式
```
tkprof D:\11G\APP\ORACLE\diag\rdbms\xe\xe\trace\xe_ora_5140.trc E:\xe_ora_5140.trc
```
![10046](https://gitee.com/henry-chr/Uplearning/raw/main/Read/Oracle/knowledge/image/10046%E4%BA%8B%E4%BB%B6.png)

#### 10053事件

**10053事件提供了Oracle如何选择执行计划，为什么会得到这样的执行计划信息。**

**10053事件的trace文件，只能阅读原始的trace文件，不能使用tkprof工具来处理。**

* 10053事件级别
    1. Level 2

        1. Column statistics
        2. Single Access Paths
        3. Join Costs
        4. Table Joins Considered
        5. Join Methods Considered (NL/MS/HA)
    2. Level 1

        1. Parameters used by the optimizer
        2. Index statistics
        3. Column statistics
        4. Single Access Paths
        5. Join Costs
        6. Table Joins Considered
        7. Join Methods Considered (NL/MS/HA)
* 执行步骤
    1. 启用10053事件
    ```
    ALTER SESSION SET EVENTS='10053 trace name context forever, level 1';
    ```
    2. 执行目标SQL
    ```
    select * from emp;
    ```
    3. 确定trace 文件
    ```
    SELECT D.VALUE || '\' || LOWER(RTRIM(I.INSTANCE, CHR(0))) || '_ora_' ||P.SPID || '.trc' AS "trace_file_name"
  FROM (SELECT P.SPID
          FROM V$MYSTAT M, V$SESSION S, V$PROCESS P
         WHERE M.STATISTIC# = 1
           AND S.SID = M.SID
           AND P.ADDR = S.PADDR) P,
       (SELECT T.INSTANCE
          FROM V$THREAD T, V$PARAMETER V
         WHERE V.NAME = 'thread'
           AND (V.VALUE = 0 OR T.THREAD# = TO_NUMBER(V.VALUE))) I,
       (SELECT VALUE FROM V$PARAMETER WHERE NAME = 'user_dump_dest') D;
    ```
    4. 关闭10053事件
    ```
    ALTER SESSION SET EVENTS '10053 trace name context off';
    ```

### AWR报告Statspack报告

### 一些现成的脚本